# ErgoVR- Object selection strategies revealed by gaze movement in VR

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils.utils import *
from datetime import date
import os

In [3]:
DATA_PATH = '../experiment_et_data/'
META_DATA_PATH = '../experiment_meta/'
PROCESSED_DATA_PATH = '../processed_data/EIH/'
os.makedirs(os.path.dirname(PROCESSED_DATA_PATH), exist_ok=True)

## Read .etd files and save as df

In [ ]:
#execution time ~6h so run only if restarting 
read_preprocess_save(DATA_PATH, PROCESSED_DATA_PATH)

## Calculate gaze information

In [4]:
samples_df = pd.read_csv(
    f'{PROCESSED_DATA_PATH}/01_ET_samples_EIH_transform.csv',
    parse_dates=['timestamp_dt'], 
    index_col=False,
    infer_datetime_format=True,
)

C:\Users\akeshava\.conda\envs\conda_ergovr\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
samples_df.columns

Index(['timestamp_dt', 'timestamp', 'leftEye_position_x', 'leftEye_position_y',
       'leftEye_position_z', 'leftEye_direction_x', 'leftEye_direction_y',
       'leftEye_direction_z', 'leftEye_raycastHitObject',
       'leftEye_raycastHitLocation_x', 'leftEye_raycastHitLocation_y',
       'leftEye_raycastHitLocation_z', 'rightEye_position_x',
       'rightEye_position_y', 'rightEye_position_z', 'rightEye_direction_x',
       'rightEye_direction_y', 'rightEye_direction_z',
       'rightEye_raycastHitObject', 'rightEye_raycastHitLocation_x',
       'rightEye_raycastHitLocation_y', 'rightEye_raycastHitLocation_z',
       'combinedEye_position_x', 'combinedEye_position_y',
       'combinedEye_position_z', 'combinedEye_direction_x',
       'combinedEye_direction_y', 'combinedEye_direction_z',
       'combinedEye_raycastHitObject', 'combinedEye_raycastHitLocation_x',
       'combinedEye_raycastHitLocation_y', 'combinedEye_raycastHitLocation_z',
       'nosePointer_position_x', 'nosePointer_

In [5]:
samples_df[['EIH_pos_x','EIH_pos_y','EIH_pos_z', 'EIH_dir_x', 'EIH_dir_y', 'EIH_dir_z']].dtypes

EIH_pos_x    1
EIH_pos_y    1
EIH_pos_z    1
EIH_dir_x    1
EIH_dir_y    1
EIH_dir_z    1
dtype: int64

### Calculate eye angular velocity

In [6]:
samples_df = calculate_EIH_theta(samples_df)
samples_df = calculate_EIH_angular_velocity(samples_df)

### Eye angular acceleration

In [7]:
samples_df =  calculate_EIH_angular_acceleration(samples_df)

In [8]:
samples_df.to_csv(
    f'{PROCESSED_DATA_PATH}/02_ET_samples_vel_acc.csv',
    index=False,
)

### Resample data

In [ ]:
samples_df = pd.read_csv(
    f'{PROCESSED_DATA_PATH}/02_ET_samples_vel_acc.csv',
    parse_dates=['timestamp_dt'], 
    index_col=False,
    infer_datetime_format=True,
)

In [ ]:
samples_df = downsample_data(samples_df)

In [ ]:
samples_df.to_csv(
    f'{PROCESSED_DATA_PATH}/03_ET_samples_downsampled_75Hz.csv',
    index=False,
)

### Label fixation-based samples

In [ ]:
samples_df = pd.read_csv(
    f'{PROCESSED_DATA_PATH}/03_ET_samples_downsampled_75Hz.csv',
    parse_dates=['timestamp_dt'], 
    index_col=False,
    infer_datetime_format=True,
)

In [16]:
samples_df.head()

,timestamp_dt,timestamp,leftEye_position_x,leftEye_position_y,leftEye_position_z,leftEye_direction_x,leftEye_direction_y,leftEye_direction_z,leftEye_raycastHitObject,leftEye_raycastHitLocation_x,...,head_acc_v,isHeadStable,isFixV,fix_duration,isOutlierFix,isOutlierSac,fix_onset,fix_stop,fix_onset_bool,fix_stop_bool
0,1970-01-01 00:00:00.360527635,0.360528,0.030739,0.001663,-0.025344,-0.034012,0.037598,0.998714,Back,-0.001267,...,-43.032384,True,True,0.33177,False,False,1970-01-01 00:00:00.360527635,1970-01-01 00:00:00.692297995,True,False
1,1970-01-01 00:00:00.370871246,0.370871,0.030738,0.001659,-0.025339,-0.033859,0.037567,0.998720,Back,-0.000663,...,-29.448556,True,True,0.33177,False,False,1970-01-01 00:00:00.360527635,1970-01-01 00:00:00.692297995,False,False
2,1970-01-01 00:00:00.381479651,0.381480,0.030734,0.001650,-0.025301,-0.033920,0.037476,0.998722,Back,0.000125,...,140.880292,True,True,0.33177,False,False,1970-01-01 00:00:00.360527635,1970-01-01 00:00:00.692297995,False,False
3,1970-01-01 00:00:00.392510742,0.392511,0.030728,0.001636,-0.025291,-0.035263,0.039078,0.998614,Back,-0.002236,...,-95.151273,True,True,0.33177,False,False,1970-01-01 00:00:00.360527635,1970-01-01 00:00:00.692297995,False,False
4,1970-01-01 00:00:00.403688431,0.403688,0.030728,0.001633,-0.025283,-0.035248,0.039414,0.998601,Back,-0.001497,...,8.569240,True,True,0.33177,False,False,1970-01-01 00:00:00.360527635,1970-01-01 00:00:00.692297995,False,False


In [9]:
samples_df = get_fixation_samples(samples_df)

### Calculate fixation durations

In [14]:
samples_df = get_fixation_duration(samples_df)

In [15]:
samples_df.to_csv(f'{PROCESSED_DATA_PATH}/04_ET_samples_fixations.csv',
                 index=False)

## Calculate grasp information

In [ ]:
samples_df = pd.read_csv(
    f'{PROCESSED_DATA_PATH}/04_ET_samples_fixations.csv',
    parse_dates=['timestamp_dt'], 
    dtype = {'isFixV':np.bool,'isOutlierFix':np.bool,'isOutlierSac':np.bool},
    infer_datetime_format=True,
)

### Calculate grasp onset, grasp stop and grasp duration

In [18]:
samples_df = get_grasp_info(samples_df)

### Calculate grasp pickup and drop-off locations

In [19]:
samples_df = get_pickup_dropoff_loc(samples_df, META_DATA_PATH)

D:\projects\ErgoVR\code\utils\utils.py:637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grasp_object = grasp.handData_graspedObject.split('_')[0]


In [20]:
samples_df.to_csv(f'{PROCESSED_DATA_PATH}/05_ET_samples_graspInfo.csv',
                 index=False)

In [ ]:
display(samples_df.columns)
samples_df['EIH_pos_x'] = np.nan
samples_df['EIH_pos_y'] = np.nan
samples_df['EIH_pos_z'] = np.nan
samples_df['EIH_dir_x'] = np.nan
samples_df['EIH_dir_y'] = np.nan
samples_df['EIH_dir_z'] = np.nan
samples_df.drop(columns=['combined_eye_theta_h', 'combined_eye_theta_v',
       'combined_eye_vel_h', 'combined_eye_vel_v', 'combined_eye_acc_h',
       'combined_eye_acc_v', 'eye_angular_vel', 'head_angular_vel',
       'isHeadStable', 'isFixV', 'fix_duration', 'corrected_fix_duration',
       'fix_onset', 'isOutlierFix', 'isOutlierSac', 'grasp_onset',
       'grasp_stop', 'grasp_duration',],
        inplace=True
)
display(samples_df.columns)

In [ ]:
samples_df.to_csv(
    f'{PROCESSED_DATA_PATH}/01_ET_samples_EIH_transform.csv',
    index=False,
)